<a href="https://colab.research.google.com/github/sooramahesh/NLP_LAB_ASSIGNMENTS/blob/main/2203A51159_SooraMahesh_lab8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

In [2]:
!pip install transformers torch optuna
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.0 MB/s eta 0:00:00


In [3]:
model_name = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
tokenizer.pad_token = tokenizer.eos_token

text_data = """
Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the woods, she met a big bad wolf who wanted to eat her....
"""
inputs = tokenizer(text_data, return_tensors="pt", truncation=True, padding=True)

In [5]:
epochs_list = [20, 60, 70]
learning_rate = 5e-5
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [6]:
def train_model(model, inputs, epochs):
    model.train()
    for epoch in range(epochs):
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")

In [7]:
for epochs in epochs_list:
    print(f"\nTraining for {epochs} epochs:")
    train_model(model, inputs, epochs)



Training for 20 epochs:
Epoch 1/20, Loss: 2.995576858520508
Epoch 2/20, Loss: 2.2445766925811768
Epoch 3/20, Loss: 1.928494930267334
Epoch 4/20, Loss: 1.459779143333435
Epoch 5/20, Loss: 1.1356196403503418
Epoch 6/20, Loss: 1.0219237804412842
Epoch 7/20, Loss: 0.7212245464324951
Epoch 8/20, Loss: 0.5809800624847412
Epoch 9/20, Loss: 0.3708662986755371
Epoch 10/20, Loss: 0.27417951822280884
Epoch 11/20, Loss: 0.16904835402965546
Epoch 12/20, Loss: 0.1292496919631958
Epoch 13/20, Loss: 0.09225475043058395
Epoch 14/20, Loss: 0.12854832410812378
Epoch 15/20, Loss: 0.03774811327457428
Epoch 16/20, Loss: 0.24154900014400482
Epoch 17/20, Loss: 0.06696135550737381
Epoch 18/20, Loss: 0.21409612894058228
Epoch 19/20, Loss: 0.014072375372052193
Epoch 20/20, Loss: 0.013311457820236683

Training for 60 epochs:
Epoch 1/60, Loss: 0.009593001566827297
Epoch 2/60, Loss: 0.010518163442611694
Epoch 3/60, Loss: 0.008963538333773613
Epoch 4/60, Loss: 0.008293229155242443
Epoch 5/60, Loss: 0.01245402637869

In [8]:
def generate_text(seed_text, max_length=50):
    input_ids = tokenizer(seed_text, return_tensors="pt").input_ids
    model.eval()
    with torch.no_grad():
        generated_text_samples = model.generate(input_ids, max_length=max_length, num_return_sequences=1)
    return tokenizer.decode(generated_text_samples[0], skip_special_tokens=True)

In [9]:
seed_text = "Once upon a time"
generated_text = generate_text(seed_text)
print("\nGenerated Text:", generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



Generated Text: Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the


In [10]:
def objective(trial):
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-4)
    epochs = trial.suggest_int('epochs', 10, 80)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    model.train()
    for epoch in range(epochs):
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    return loss.item()

In [11]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)

print("\nBest hyperparameters found:")
print(study.best_params)

[I 2024-11-05 04:38:31,251] A new study created in memory with name: no-name-a4d4d6b1-951b-4250-8c06-047eafc26100
<ipython-input-10-f5ef70386882>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-4)
[I 2024-11-05 04:40:17,086] Trial 0 finished with value: 1.8997638107975945e-05 and parameters: {'learning_rate': 4.380956784133458e-05, 'epochs': 41}. Best is trial 0 with value: 1.8997638107975945e-05.
[I 2024-11-05 04:42:46,051] Trial 1 finished with value: 2.819141445797868e-05 and parameters: {'learning_rate': 9.015349119183978e-05, 'epochs': 65}. Best is trial 0 with value: 1.8997638107975945e-05.
[I 2024-11-05 04:43:36,904] Trial 2 finished with value: 2.039806349785067e-05 and parameters: {'learning_rate': 7.607695882726798e-05, 'epochs': 22}. Best is trial 0 


Best hyperparameters found:
{'learning_rate': 9.525552598938056e-05, 'epochs': 28}
